<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/07_Ajuste_por_Instrucciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instruction Tuning: Una Guía Práctica

En este notebook aprenderemos sobre el ajuste por instrucciones (instruction tuning) de una manera práctica y sencilla.


## 1) Configuración del Entorno
 Primero, importamos las bibliotecas necesarias y configuramos nuestro entorno.


In [3]:
# Instalamos las bibliotecas necesarias
print("Instalando bibliotecas necesarias...")
!pip install --quiet transformers datasets torch seqeval

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
import pandas as pd
import numpy as np

# Verificamos si tenemos GPU disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo disponible: {device}")

Instalando bibliotecas necesarias...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Dispositivo disponible: cpu


## 2) Preparación de Datos
Creamos un conjunto simple de datos de ejemplo con instrucciones.



In [17]:
instrucciones = [
    {
        "instruccion": "Explica qué es una estrella",
        "contexto": "",
        "respuesta": "Una estrella es un cuerpo celeste masivo que produce luz y calor mediante reacciones nucleares en su núcleo. El Sol es un ejemplo de estrella."
    },
    {
        "instruccion": "Define qué es un planeta",
        "contexto": "",
        "respuesta": "Un planeta es un cuerpo celeste que orbita alrededor de una estrella, tiene suficiente masa para ser esférico y ha limpiado su órbita de otros objetos."
    },
    {
        "instruccion": "Explica qué es la fotosíntesis",
        "contexto": "",
        "respuesta": "La fotosíntesis es el proceso por el cual las plantas convierten la luz solar en energía química, produciendo oxígeno y glucosa a partir de agua y dióxido de carbono."
    },
    {
        "instruccion": "Describe qué es un átomo",
        "contexto": "",
        "respuesta": "Un átomo es la unidad más pequeña de la materia que mantiene las propiedades de un elemento químico. Está compuesto por un núcleo con protones y neutrones, rodeado por electrones."
    },
    {
        "instruccion": "Explica qué es la gravedad",
        "contexto": "",
        "respuesta": "La gravedad es una fuerza fundamental de la naturaleza que hace que los objetos con masa se atraigan entre sí. Es la fuerza que nos mantiene en la Tierra y hace que los planetas orbiten alrededor del Sol."
    }
]

# Convertimos a Dataset de HuggingFace
dataset = Dataset.from_list(instrucciones)

# Mostramos un ejemplo
print("Ejemplo del dataset:")
print(dataset[0])

Ejemplo del dataset:
{'instruccion': 'Explica qué es una estrella', 'contexto': '', 'respuesta': 'Una estrella es un cuerpo celeste masivo que produce luz y calor mediante reacciones nucleares en su núcleo. El Sol es un ejemplo de estrella.'}


## 3) Carga del Modelo Base
 Utilizaremos un modelo pequeño para nuestras pruebas.

In [31]:
# Cargamos el modelo y el tokenizador
modelo_nombre = "gpt2"  # Usamos GPT-2 por su tamaño reducido
print(f"Cargando modelo: {modelo_nombre}")

tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
model = AutoModelForCausalLM.from_pretrained(modelo_nombre)

# Configuración básica del tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print("Modelo y tokenizer cargados correctamente")

Cargando modelo: facebook/opt-350m


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Modelo y tokenizer cargados correctamente


## 4) Preparación de los Datos
Formateamos nuestras instrucciones para el entrenamiento.

In [32]:
def formatear_instruccion(ejemplo):
    """
    Formatea cada ejemplo en un formato consistente
    """
    return f"""
### Instrucción: {ejemplo['instruccion']}
### Contexto: {ejemplo['contexto']}
### Respuesta: {ejemplo['respuesta']}
"""

# Mostramos un ejemplo formateado
ejemplo_formateado = formatear_instruccion(dataset[0])
print("Ejemplo formateado:")
print(ejemplo_formateado)

Ejemplo formateado:

### Instrucción: Explica qué es una estrella
### Contexto: 
### Respuesta: Una estrella es un cuerpo celeste masivo que produce luz y calor mediante reacciones nucleares en su núcleo. El Sol es un ejemplo de estrella.



## 5) Tokenización de Datos
Convertimos nuestros textos en tokens que el modelo puede procesar.


In [33]:
def tokenizar_datos(ejemplo):
    """
    Tokeniza un ejemplo formateado y prepara los labels para el entrenamiento
    """
    # Tokenizamos el texto completo
    texto_completo = formatear_instruccion(ejemplo)
    encodings = tokenizer(
        texto_completo,
        truncation=True,
        max_length=128,
        padding="max_length"
    )

    # Los labels son los mismos que input_ids para entrenamiento de lenguaje
    encodings['labels'] = encodings['input_ids'].copy()

    return encodings

# Tokenizamos el dataset
dataset_tokenizado = dataset.map(tokenizar_datos)
print("Dataset tokenizado correctamente")

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset tokenizado correctamente


## 6) Configuración del Entrenamiento
Definimos los parámetros para el ajuste del modelo.

In [34]:
from transformers import TrainingArguments

args_entrenamiento = TrainingArguments(
    output_dir="./modelo_ajustado",
    num_train_epochs=5,
    per_device_train_batch_size=2,
    learning_rate=5e-5,
    logging_steps=5,
    report_to="none",
    eval_strategy="no"  # No hacemos evaluación en este ejemplo simple
)

## 7) Entrenamiento del Modelo

Realizamos el ajuste del modelo con nuestros datos.


In [35]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args_entrenamiento,
    train_dataset=dataset_tokenizado,
)

# Iniciamos el entrenamiento
print("Comenzando el entrenamiento...")
trainer.train()

Comenzando el entrenamiento...


Step,Training Loss
5,3.104600
10,0.698400
15,0.301600


TrainOutput(global_step=15, training_loss=1.3681646505991618, metrics={'train_runtime': 417.5949, 'train_samples_per_second': 0.06, 'train_steps_per_second': 0.036, 'total_flos': 5824472678400.0, 'train_loss': 1.3681646505991618, 'epoch': 5.0})

## 8) Prueba del Modelo

Probamos nuestro modelo ajustado con nuevas instrucciones.


In [36]:
def probar_modelo(instruccion, contexto=""):
    """
    Prueba el modelo con una nueva instrucción
    """
    # Formateamos el prompt incluyendo un marcador claro para la respuesta
    prompt = f"""### Instrucción: {instruccion}
### Contexto: {contexto}
### Respuesta: La respuesta a esta instrucción es:"""

    # Tokenizamos el prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)

    # Generamos la respuesta con parámetros más conservadores
    outputs = model.generate(
        inputs["input_ids"],
        max_length=150,          # Longitud máxima razonable
        min_length=30,           # Forzamos una respuesta mínima
        temperature=0.5,         # Temperatura más conservadora
        do_sample=True,
        top_p=0.85,
        top_k=40,
        no_repeat_ngram_size=3,  # Evitamos más repeticiones
        num_beams=3,             # Añadimos beam search
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    # Decodificamos y limpiamos la respuesta
    respuesta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extraemos solo la parte después de nuestro marcador
    try:
        respuesta = respuesta_completa.split("La respuesta a esta instrucción es:")[-1].strip()
        return respuesta if respuesta else "No se generó una respuesta válida."
    except:
        return "Error al procesar la respuesta."

# Probamos con una nueva instrucción
nueva_instruccion = "Explica qué es una estrella"
respuesta = probar_modelo(nueva_instruccion)
print(f"Instrucción: {nueva_instruccion}")
print(f"Respuesta: {respuesta}")

Instrucción: Explica qué es una estrella
Respuesta: Una estrellana es un cuerpo celeste masivo que produce un núcleo con una fuerza química. El Sol es un ejemplo de estrella.
▄ {: [| ================================= "$:/aditional>>\ Philips Hue lightens up to match the surrounding area's temperature and produces a white light that is emitted by a set of LEDs that are connected to a power source.
